<img src="https://cursos.virtual.uniandes.edu.co/isis4219/wp-content/uploads/sites/162/2014/11/cropped-misisheader.png" ><br>
# Machine Learning Techniques - ISIS4219

Intersemestral 2020

### Integrantes:
* Juan José Villegas
* Juan Sebastián Méndez

De acuerdo a los conceptos vistos en clase desarrolle el siguiente ejercicio práctico:

 1. El Internet de las Cosas (IoT) está desempeñando un papel cada vez más importante en nuestras actividades cotidianas, al habilitar la conexión de los objetos que nos rodean con servicios digitales. 
 
 Últimamente, IoT se ha visto afectado por una variedad de botnets, los cuales intentan explotar vulnerabilidades en los protocolos de aplicación que interactúan directamente con los sistemas, provocando violaciones de seguridad. 
 
 Una solución a este tipo de ataques es identificar, de manera automática, eventos maliciosos contra los protocolos de comunicación. En este sentido, ***se desea desarrollar un sistema de detección de intrusos que permita una identificación temprana de estas amenazas***. Sin embargo, el Departamento de Seguridad quiere identificar que ataque específico se está ejecutando para poder implementar los protocolos correspondientes.

Como recomendación general, se espera que para la entrega del taller se aplique la estructura establecida por la metodología CRISP- DM, según corresponda con las caracteristicas de cada conjunto de datos.

## 1. Importación de las librerías

In [51]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

## 2. Perfilamiento y entendimiento de los datos

In [52]:
# Importando los datos
train = pd.read_csv('./Punto 1/UNSW_NB15_training-set.csv', encoding = "ISO-8859-1")
test = pd.read_csv('./Punto 1/UNSW_NB15_testing-set.csv', encoding = "ISO-8859-1")

In [53]:
train.head()

,ï»¿id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [38]:
#profile = ProfileReport(train)

In [39]:
#profile.to_file("profile_report.html")

### Notas

* El label no parece ser necesario porque lo que se quiere as predicir el tipo de ataque y hay un tipo que es normal y equivale al label 0. Podría ser eliminado
* Hay que tranformar los atributos categóricos **proto**, **service** y **state** a números
* Hay que escalar todas las variable ya que están muy dispersas

## 3. Limpieza y preparación de datos

In [40]:
# Drop features no needed
# We should think of droping highly correlated features 
features_to_drop = ['label', 'attack_cat', 'ï»¿id', 'sbytes', 'dbytes', 'dloss', 'sloss']
X_train, y_train = train.drop(features_to_drop, axis=1), train['attack_cat']
X_test, y_test = test.drop(features_to_drop, axis=1), test['attack_cat']

In [41]:
X_train.head()

,dur,proto,service,state,spkts,dpkts,rate,sttl,dttl,sload,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,0.000011,udp,-,INT,2,0,90909.0902,254,0,180363632.0,...,1,1,1,2,0,0,0,1,2,0
1,0.000008,udp,-,INT,2,0,125000.0003,254,0,881000000.0,...,1,1,1,2,0,0,0,1,2,0
2,0.000005,udp,-,INT,2,0,200000.0051,254,0,854400000.0,...,1,1,1,3,0,0,0,1,3,0
3,0.000006,udp,-,INT,2,0,166666.6608,254,0,600000000.0,...,2,2,1,3,0,0,0,2,3,0
4,0.000010,udp,-,INT,2,0,100000.0025,254,0,850400000.0,...,2,2,1,3,0,0,0,2,3,0


In [42]:
num_preprocessor = make_pipeline(
    StandardScaler()
)

categories_preprocessor = make_pipeline(
    OneHotEncoder(handle_unknown='ignore')
)

preprocessor = make_column_transformer(
    (num_preprocessor, ('spkts', 'dpkts', 'dinpkt', 'djit', 'trans_depth', 'response_body_len')),
    (categories_preprocessor, ('proto', 'service', 'state'))
)

#full_pipeline = make_pipeline(preprocessor, SGDClassifier())

In [43]:
X_train_prepared = preprocessor.fit_transform(X_train)

In [44]:
X_train_prepared.shape

(82332, 157)

## 4. Modelamiento

Vamos a usar el SGD classifier para probar

In [46]:
model = SGDClassifier(n_jobs=-1)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

### Sintonización de hiperparámetros

In [48]:
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [500, 1000, 10000]
}
grid_search = GridSearchCV(model, param_grid)
%time grid_search.fit(X_train_prepared, y_train)
grid_search.best_params_

CPU times: user 42.1 s, sys: 354 ms, total: 42.4 s
Wall time: 10.6 s


{'alpha': 0.001, 'max_iter': 10000}

In [49]:
grid_search.best_score_

0.7103067539110035

In [50]:
grid_search.best_estimator_

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=10000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

## 5. Predicción

## 6. Validación